In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /Users/rob/.julia/packages/Gadfly/09PWZ/src/mapping.jl:228


Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:32:01 of 0:32:03 remaining]
Chain 1:  10% [0:01:05 of 0:01:12 remaining]
Chain 1:  20% [0:01:02 of 0:01:18 remaining]
Chain 1:  30% [0:00:53 of 0:01:16 remaining]
Chain 1:  40% [0:00:44 of 0:01:13 remaining]
Chain 1:  50% [0:00:36 of 0:01:12 remaining]
Chain 1:  60% [0:00:29 of 0:01:11 remaining]
Chain 1:  70% [0:00:22 of 0:01:12 remaining]
Chain 1:  80% [0:00:14 of 0:01:12 remaining]
Chain 1:  90% [0:00:07 of 0:01:13 remaining]
Chain 1: 100% [0:00:00 of 0:01:13 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:54 of 0:00:60 remaining]
Chain 2:  20% [0:00:53 of 0:01:06 remaining]
Chain 2:  30% [0:00:50 of 0:01:11 remaining]
Chain 2:  40% [0:00:43 of 0:01:12 remaining]
Chain 2:  50% [0:00:36 of 0:01:12 remaining]
Chain 2:  60% [0:00:29 of 0:01:13 remaining]
Chain 2:  70% [0:00:22 of 0:01:12 remaining]
Chain 2:  80% [0:00:15 of 0:01:13 remaining]
Chain 2:  90% [0:00:07 of 0:01:13 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[29.9472 111.394 0.959827; 27.9366 115.383 0.871622; … ; 28.8011 114.475 0.898107; 27.6132 114.111 0.891489]

[26.629 116.476 0.847421; 28.5389 115.651 0.862257; … ; 28.8117 115.445 0.867982; 27.4584 113.715 0.911122]

[27.4771 113.923 0.904856; 27.4052 115.326 0.85804; … ; 26.1092 112.901 0.922518; 26.4789 116.154 0.844083]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean        SD        Naive SE       MCSE        ESS   
     s2  26.1748755 1.97409708 0.01201397225 0.0200520060 9000.0000
beta[1] 113.8700836 1.91107851 0.01163045342 0.0386657520 2442.8913
beta[2]   0.9052138 0.04202744 0.00025577086 0.0008422385 2489.9794

Quantiles:
            2.5%       25.0%       50.0%       75.0%      97.5%   
     s2  22.5566795  24.8085586  26.0955340  27.442354  30.3310424
beta[1] 110.0973973 112.5830749 113.8544751 115.148416 117.7649489
beta[2]   0.8209895   0.8771868   0.9053131   0.933213   0.9877224



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*